# Exercise - Text Mining - Classification - SCIKIT-LEARN

We will predict the category of a product based on the description of the product.

**The unit of analysis is a product**

In [516]:
import pandas as pd
import numpy as np

In [517]:
products = pd.read_csv('products.csv')

In [518]:
products.head(5)

,Category,Description
0,Electronics,HP 680 Original Ink Advantage Cartridge (Black...
1,Clothing & Accessories,Bold N Elegant Navy Blue Thin Summer Pregnancy...
2,Books,The Travel Book: A Journey Through Every Count...
3,Electronics,Tiny Deal Compact 10x25 Mini Binoculars Telesc...
4,Clothing & Accessories,Nimble House 16Pcs/Set Unisex Women Men No Tie...


In [519]:
products.shape

(5000, 2)

## Assign the "target" variable



In [520]:
target = products['Category']

## Assign the "text" (input) variable

In [521]:
products[['Description']].isna().sum()

Description    0
dtype: int64

In [522]:
input_data = products[['Description']]

## Split the data

In [523]:
from sklearn.model_selection import train_test_split

train_set, test_set, train_y, test_y = train_test_split(input_data, target, test_size=0.3, random_state=42)

In [524]:
train_set.shape, train_y.shape

((3500, 1), (3500,))

In [525]:
test_set.shape, test_y.shape

((1500, 1), (1500,))

# Data Prep

In [526]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import FunctionTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

## Sklearn: Text preparation

Create a pipeline to transform the text column and create a term by document matrix.

In [527]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

In [528]:
def new_col(df):
    #Create a copy so that we don't overwrite the existing dataframe
    df1 = df.copy()
    
    # First, conver the dataframe column to a numpy array. Then, call the ravel function to make it one-dimensional
    return np.array(df).ravel()

In [529]:
new_col(train_set)

array(['Dehman 100-Percent Silky Satin Hair Beauty Pillowcase, Black, King Size DEHMANSATIN Silk originated in INDIA and it is an honor to present this ancient oriental gift to you. The collective wisdom of the sericulture people of this land has been condensed DEHMAN has always adhered to its original heart from generation to generation. From raw satin silk material collection to processing and to finished products,each step is carefully selected. DEHMAN hopes that every customer will have a wonderful satin silk experience and enjoy the silky noble life. Why Choose DEHMAN Pillowcase ? Material：DEHMAN Satin Silk pillowcase is crafted by 19 momme pure mulberry silk. Both sides of the pillowcase is organic and natural satin silk.Light weight and easy to carry. Design: It is designed with hidden zipper closure.With Queen(20x30inches) and King(20x36inches) and Standard size(20x26inches) in various kinds colours. Quality：Superior durable plain color ,not easy to run after washing.Exquisite 

In [530]:
text_column = ['Description']

In [531]:
number_svd_components = 600

In [532]:
text_transformer = Pipeline(steps=[
                ('my_new_column', FunctionTransformer(new_col)),
                ('text', TfidfVectorizer(stop_words='english')),
                ('svd', TruncatedSVD(n_components=number_svd_components, n_iter=10))
            ])

In [533]:
preprocessor = ColumnTransformer([
                     ('text', text_transformer, text_column),
                    ],
        remainder='drop')

#passtrough is an optional step. You don't have to use it.

In [534]:
train_x = preprocessor.fit_transform(train_set)
train_x

array([[ 1.32127476e-01, -3.83815352e-02, -5.52127985e-02, ...,
         8.09677090e-03, -2.14349616e-02, -2.72402704e-03],
       [ 1.19672838e-03,  1.09071177e-03, -9.41521626e-06, ...,
        -2.29548546e-02,  5.18547985e-03, -3.76584220e-04],
       [ 5.74701078e-02,  5.16487292e-02,  8.81057288e-03, ...,
         2.90748205e-02, -1.04374972e-02,  1.18385381e-02],
       ...,
       [ 1.59120292e-01, -7.66943792e-02, -1.60781830e-01, ...,
        -5.69391220e-03, -1.90266612e-02, -1.32172744e-02],
       [ 8.31440381e-02, -4.33959857e-02, -3.44149656e-02, ...,
         1.20205428e-02, -2.42572489e-04,  8.64722527e-03],
       [ 9.66538621e-02, -3.95970843e-02, -1.19857668e-01, ...,
         3.43162060e-02, -2.54782154e-03, -4.07207897e-02]])

In [535]:
train_x.shape

(3500, 600)

In [536]:
# Transform the test data
test_x = preprocessor.transform(test_set)

test_x

array([[ 1.97114994e-02,  1.21761976e-02,  5.64651570e-03, ...,
        -8.13553226e-03, -8.26550234e-03,  1.00738511e-02],
       [ 8.02758518e-02, -4.06088051e-02,  2.60699499e-02, ...,
         2.31253215e-02, -1.36349166e-03,  4.31380464e-03],
       [ 1.73274443e-01, -7.02506391e-02, -4.60694990e-02, ...,
        -5.52169608e-03,  1.05558628e-03,  8.66037831e-03],
       ...,
       [ 8.83440905e-02,  3.66329274e-02,  6.27166859e-03, ...,
         1.73352126e-02, -4.35370804e-03,  6.35409616e-03],
       [ 8.82624542e-02, -3.47200813e-02,  3.56394126e-02, ...,
        -3.62596497e-03, -2.01948519e-02,  3.52528429e-03],
       [ 1.98262116e-02, -9.44310695e-03,  2.90959470e-02, ...,
         4.00044386e-03, -5.17150873e-03,  1.24539614e-04]])

In [537]:
test_x.shape

(1500, 600)

### Baseline

In [538]:
from sklearn.dummy import DummyClassifier

dummy_clf = DummyClassifier(strategy="most_frequent")

dummy_clf.fit(train_x, train_y)

DummyClassifier(strategy='most_frequent')

In [539]:
from sklearn.metrics import accuracy_score

In [540]:
#Baseline Train Accuracy
dummy_train_pred = dummy_clf.predict(train_x)

baseline_train_acc = accuracy_score(train_y, dummy_train_pred)

print('Baseline Train Accuracy: {}' .format(baseline_train_acc))

Baseline Train Accuracy: 0.38142857142857145


In [541]:
#Baseline Test Accuracy
dummy_test_pred = dummy_clf.predict(test_x)

baseline_test_acc = accuracy_score(test_y, dummy_test_pred)

print('Baseline Test Accuracy: {}' .format(baseline_test_acc))

Baseline Test Accuracy: 0.386


# Try one of the classifiers we have covered so far

### Decision Tree Classifier

In [542]:
from sklearn.tree import DecisionTreeClassifier 

tree_clf = DecisionTreeClassifier(max_depth=5)

tree_clf.fit(train_x, train_y)

DecisionTreeClassifier(max_depth=5)

In [543]:
from sklearn.metrics import accuracy_score

## Accuracy

In [544]:
#Train accuracy:
train_y_pred = tree_clf.predict(train_x)

print(accuracy_score(train_y, train_y_pred))

0.9082857142857143


In [545]:
#Test accuracy:
test_y_pred = tree_clf.predict(test_x)

print(accuracy_score(test_y, test_y_pred))

0.8953333333333333


## Generate the confusion matrix

In [546]:
from sklearn.metrics import confusion_matrix

#Test confusion matrix
confusion_matrix(test_y, test_y_pred)

array([[322,   7,  11,  25],
       [  1, 249,   4,   4],
       [ 13,   0, 253,  32],
       [ 19,  16,  25, 519]], dtype=int64)

# Try another classifier we have covered so far

### Support Vector Classifier, Kernal= 'linear'

In [547]:
from sklearn.svm import SVC
 
lin_svm = SVC(kernel="linear")

lin_svm.fit(train_x, train_y)

SVC(kernel='linear')

In [548]:
from sklearn.metrics import accuracy_score

## Accuracy

In [549]:
#Predict the train values
train_y_pred = lin_svm.predict(train_x)

#Train accuracy
accuracy_score(train_y, train_y_pred)

0.9482857142857143

In [550]:
#Predict the test values
test_y_pred = lin_svm.predict(test_x)

#Test accuracy
accuracy_score(test_y, test_y_pred)

0.9393333333333334

## Generate the confusion matrix

In [551]:
from sklearn.metrics import confusion_matrix

#test confusion matrix 
confusion_matrix(test_y, test_y_pred)

array([[334,   6,   3,  22],
       [  3, 250,   1,   4],
       [  7,   0, 261,  30],
       [  6,   4,   5, 564]], dtype=int64)

# Check for the number of components

**Determine whether increasing/decreasing the number of components increases/decreases the two models' accuracies** 
     
  Discuss your findings below.